In [5]:
import numpy as np

import cv2
import time

import imutils

from directkeys import  W, A, S, D
from directkeys import PressKey, ReleaseKey 

from imutils.video import FileVideoStream, VideoStream

path_vid = 'Material/Videos/full.mp4'

In [6]:
size_height = 600
size_width = 600
tracker_type = 'CSRT'

In [17]:
def draw(ret, bbox, frame):
         # Draw bounding box
    if ret:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Failed in Tracking", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)


def get_centroid(bbox): #center point of rectangle
    x, y, w, h = bbox
    centx = int(x + w // 2)
    centy = int(y + h // 2)
    return (centx, centy)

def drawbox(ret, bbox, frame): #draws rectangle from bbox
    global q
    if ret:
        #Tracking Success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)

    if p1[0] < 10: #quits program if taken hand to left top corner
        q = True # quit

In [9]:
key_steer, key_acc = None, None #key to accelerate

# delay = 0.3
last = 0 #last extreme of steer
chng = 3 #change in last, to remove small changes fulucations
chngt = 3 #to restart change to 0, if moved towards centre by some value

COLOR_RED = (0, 0, 255)
COLOR_BLACK = (0, 0, 0)

THRESH_ACTION = 60 # threshold for a action to be registered, inside this no movement

def steer(bbox): #to steer left and right
    global last
    x, _ = get_centroid(bbox)
    diff = x - centleft[0] 
    
    if abs(diff) < last: 
        if abs(diff) < last-chng*chngt:
            last = 0
        return
    
    last = abs(diff) + chng # saving the last extreme, to help with random noise movements
    
    if abs(diff) < THRESH_ACTION:
            return

    if diff > 0:
        key_steer = D
    else:
        key_steer = A
        
    PressKey(key_steer)
    return diff    
    
def accelerate(bbox):
    # global key_acc
    
    _, y = get_centroid(bbox)
    diff = - (y - centright[1])  # '-' as y decr upward

    if abs(diff) < THRESH_ACTION:
        return
    
    if diff > 0:
        key_acc = W
    else:
        key_acc = S
        
    PressKey(key_acc)
    return diff


def draw_circle(frame, center, radius=THRESH_ACTION, color=COLOR_BLACK, thickness=3):
    cv2.circle(frame, center, radius, color, thickness)
 
def action(bboxleft, bboxright):    
    diff_acc = accelerate(bboxright) 
    diff_steer =steer(bboxleft) 
    return diff_acc, diff_steer

In [10]:
def get_frame(): # to get frame by reading fvs, init fvs first
    frame = fvs.read()
    if frame is None:
        raise Exception("Frame Not Found")
        return
    frame = cv2.flip(frame, 1)
    frame = imutils.resize(frame, width=size_width, height=size_height) 
    return frame

In [12]:
fvs = FileVideoStream(0).start() #0 for web cam
time.sleep(2.0) #to allow web cam to open

TIMER_SETUP = 3 # timer for capturing base image
t = time.time()

while True:
    frame = get_frame()
    curr = (time.time() - t)
    if curr > TIMER_SETUP:
        break
    cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
    cv2.imshow("Setup", frame)
    cv2.waitKey(1)

FRAME = frame.copy()
cv2.destroyAllWindows()

#Make box around hand
frame = FRAME.copy()
cv2.putText(frame, 'Select Left Hand for steering', (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_RED, 2)
bboxleft = cv2.selectROI(frame, False) # bounding box for left hand 

frame = FRAME.copy()
cv2.putText(frame, 'Select Right Hand for acceleration', (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_RED, 2)
bboxright = cv2.selectROI(frame, False) # bounding box for right hand

centleft = get_centroid(bboxleft)
centright = get_centroid(bboxright)

draw_circle(frame, centleft) # circle, outside this movements will happend
draw_circle(frame, centright)

BBL, BBR = bboxleft, bboxright # saving it for later

cv2.destroyAllWindows()

In [13]:
bboxleft, bboxright = BBL, BBR 

# Creating the CSRT Tracker
trackerleft = cv2.TrackerCSRT_create() # left hand for steering
trackerright = cv2.TrackerCSRT_create() # right hand for acceleration (acc)

# Initialize tracker with first frame and bounding box
trackerleft.init(FRAME, bboxleft)
trackerright.init(FRAME, bboxright)

cv2.putText(frame.copy(), 'Put both your hands in Postion', (100,70), \
    cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)

TIMER_SETUP = 3 # put hands in boxes
t = time.time()

while True:
    frame = get_frame()
    curr = (time.time() - t)
    if curr > TIMER_SETUP or frame is None:
        break
    cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
    drawbox(True, bboxleft, frame)
    drawbox(True, bboxright, frame)
    cv2.imshow("Tracking", frame)
    cv2.waitKey(1)

cv2.destroyAllWindows()

#Open the game, before timer stops

q = False #to quit, take your hand on top corner

while True:
    frame = get_frame()
    
    if frame is None:
        break
        
#     frame = cv2.flip(frame, 1)
#     frame = imutils.resize(frame, width=size_width, height=)  

    retleft, bboxleft = trackerleft.update(frame) # retleft - True, if found hand
    retright, bboxright = trackerright.update(frame)
    
    if not retleft == retright == True:
        print("Tracking stopped")
        break
    
    diff_acc, diff_steer  = action(bboxleft, bboxright)
    
    cv2.putText(frame, "Acceleration -> " + str(diff_acc), (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)
    cv2.putText(frame, "Steer -> " + str(diff_steer), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)
     
    drawbox(retleft, bboxleft, frame)
    drawbox(retright, bboxright, frame)

    draw_circle(frame, centleft)
    draw_circle(frame, centright)

#     write('')

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
    cv2.imshow("Tracking", frame)
    
    k = cv2.waitKey(1)

    if k == 13 or q: #13 is the Enter Key
        break
    
cv2.destroyAllWindows()
fvs.stop()